In [1]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.3 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 258 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 409 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 318 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 857 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 2.9 MB/s eta 0:00:01    |████████                        | 2.8 MB 2.7 MB/s eta 0:00:04
     |████████████████████████████████| 8.1 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 2.7 MB/s eta 0:00:011
     |████████████████████████████████| 

In [2]:
import gradio as gr

/Users/yedijamessa/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/yedijamessa/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)
demo = gr.Interface(
    fn=greet,
    inputs=["text","slider"],
    outputs=["text"],
)
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import gradio as gr
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

In [6]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [7]:
def generate_caption(image):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model.generate(**inputs)
    caption = processor.decode(outputs[0],skip_special_tokens=True)
    return caption

In [10]:
def caption_image(image):
    try: 
        caption = generate_caption(image)
        return caption
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [11]:
iface = gr.Interface(
    fn=caption_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Captioning with BLIP",
    description="Upload an image to generate a caption."
)

iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


/Users/yedijamessa/Library/Python/3.9/lib/python/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## Image Classification in PyTorch

In [12]:
import torch

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True).eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /Users/yedijamessa/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/yedijamessa/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:12<00:00, 3.83MB/s]


In [13]:
import requests
from PIL import Image
from torchvision import transforms

In [14]:
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

In [19]:
def predict(inp):
 inp = transforms.ToTensor()(inp).unsqueeze(0)
 with torch.no_grad():
  prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
  confidences = {labels[i]: float(prediction[i]) for i in range(1000)}
 return confidences

In [20]:
gr.Interface(fn=predict,
       inputs=gr.Image(type="pil"),
       outputs=gr.Label(num_top_classes=3),
       examples=["/content/lion.jpg", "/content/cheetah.jpg"]).launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
